# Basic I/O

## Outline

* Intro to impure functions
* Basic IO actions
* Composing IO actions
* The do block
* Recap

In this section we will introduce the concepts of IO actions. We will begin with introducing these functions and then construct an intuative but cumbersome way of composing these function. Then we will introduce a more programmer friendly way of composing them that hides this awkward construction. At the end we will recap the import concepts.

## Intro to impure functions
Haskell is a purely functional language. This means that for every function **only** the arguments determine what the outcome will be, it does not change any state. But this is quite a narrow scope for a program. In the real world we want application to process data from the outside and provide results back to the outside world, it is interactive and should change state outside the scope of the program.

The interactions and changes to state outside this scope are called **side effects**. Say we have a function that fetches the time and logs it into a file. Each time we call this function we give it no inputs and get no outputs back, we only observe the side effect of the file that is changed. This function is thus **impure**. As a recap

| Pure | Impure |
| --- | --- |
| Always produces the same result when given the same parameter | May produce different results for the same parameter |
| Never has side effect | May have side effects |
| Never alters state | May alter the global state of the program, system, or world |

## Basic IO actions

Haskell strictly seperates pure code from the impure world. But we would still like our programs to alter the world outside of our program, it does this via **IO actions** denoted by `IO a`. All these impure function include a type that is marked with the `IO` wrapper followed by some type `a`. This marks that it first performs some computation with possibly side effect and then return something of type `a`.

As a first example we have the `putStrLn` function which has as input the string `"Hello World"` and output type `()`. Together the resulting type is of type `IO ()`. The `()` type is the empty tuple, since it has zero elements it can only be one thing, namely nothing.

In [2]:
:t putStrLn --(takes some string and writes it to StdOut)

-- An IO action that perform the action of printing "Hello World".
main :: IO ()
main = putStrLn "Hello World"
main

putStrLn :: String -> IO ()

Hello World

Note that these `IO a` wrapped function are still functions and can thus be composed with other functions, just keep in mind that the type signatures you are using are correct. Now since a combination of an impure function and a pure function still has side effects the composed function also is a impure function and thus uses the wrapped marked `IO`. We thus conclude that we can **never** escape the `IO` wrapper once we used it.

In [8]:
-- A pure function that prepends the string "Hello " to another string
addHello :: String -> String
addHello string = "Hello " ++ string

-- Using the putStrLn function in the definition of an IO action.
newPutStrLn :: String -> IO ()
newPutStrLn name = putStrLn (addHello name)
newPutStrLn "John"

Hello John

Another example that requires no inputs but fetches the time as a side effect is the function`getCurrentTime` in the module `Data.Time.Clock`.

In [1]:
import Data.Time.Clock (getCurrentTime) -- Importing some library for the getCurrentTime function

-- Fetching the current time with an action and getting the type of this function
:t getCurrentTime
getCurrentTime

getCurrentTime :: IO UTCTime

2022-06-17 17:34:10.542792817 UTC

In [1]:
import System.Directory (listDirectory) -- Importing some library for the listDirectory function

-- this IO action will list the content of the directory it is given in a list. The "." input refers to the current jupyter working directory (week02)
:t listDirectory
listDirectory "."

listDirectory :: FilePath -> IO [FilePath]

["2.2-Creating-Types.ipynb","2.3-Creating-Type-Classes.ipynb","2.1-Intro-to-Type-Classes.ipynb","2.4-Basic-IO(2)(3).ipynb",".ipynb_checkpoints"]

## Composing IO actions
Now that we can use actions to do usefull side effects outside of our program we would like to be able to compose two actions, just like with pure functions. This should not be hard, just use another `IO a` as an input as before to get a function of type `IO a -> IO b`. But there is a problem with this construction, this is highlighted in the below example.

In [76]:
-- An IO action that perform the action of printing "Hello". 
someIOType :: IO ()
someIOType = putStrLn "Hello"

-- A naive and wrong function that can combine two IO () actions
combineIO :: IO () -> IO ()    -- A naive and wrong way of composing IO actions
combineIO io = putStrLn "world"
combineIO someIOType

world

Here the `someIOType` which perform the action of writing "Hello" is not being performed. This is because the input `io` is not being used after the equal sign, and thus not being evaluated. But how can we call this 'io' input on the right side? To solve this we introduce the **then** operator given by `>>` and the **bind** operator given by `>>=`. The difference between the two is that the former operator does not use the output of the first operation for the second one where the latter operator does. So for the then and bind operator we get

In [8]:
import System.Directory (listDirectory)

-- A new expression that is of type IO ()
someIOType :: IO ()
someIOType = putStrLn "Hello" 

-- A function that can combine two IO () actions
combineIOThen :: IO () -> IO ()
combineIOThen io = io >> putStrLn "world"
combineIOThen someIOType

-- A pure function that takes as input a list of file paths and return the first one as a string
headFilePathList :: [FilePath] -> String
headFilePathList = show . head

-- A new expression that is of type IO [FilePath]
someIOTypeTwo :: IO [FilePath]
someIOTypeTwo = listDirectory "."

-- A function that can combine two IO action and use the the wrapped output of the first action as an input of the first
combineIOWithBind :: IO [FilePath] -> IO ()
combineIOWithBind io = io >>= putStrLn . headFilePathList
combineIOWithBind someIOTypeTwo

Hello
world

"2.2-Creating-Types.ipynb"

## The do block

Now that we know how to compose actions we are going to generalize this with the so called **do block**. To introduce the necessity and reasons of this we look at a few examples. First consider the composition of many actions via the then operator `>>`.

In [27]:
import System.Directory (listDirectory)

--  IO () action        IO [FilePath]           IO ()
putStrLn "Action 1" >> listDirectory "." >> putStrLn "Action 3"

Action 1
Action 3

Note that each action is **independent** of the others, the first and last output `Action 1` and `Action 3` respectivly. The middle one grabs the current files in the directory "." which we do not output. Thus we see that each action can be composed without thinking about if the composition is type correct. Lets look at how this pattern would work for the bind operator `>>=`.

In [15]:
import System.Directory (listDirectory)

-- A new expression that is of type IO ()
someIOType :: IO ()
someIOType = putStrLn "Hello" 

-- A pure function that takes as input a list of file paths and return the first one as a string
headFilePathList :: [FilePath] -> String
headFilePathList = show . head

-- A pure function that takes as input a list of file paths and return the first one as a string
lastFilePathList :: [FilePath] -> String
lastFilePathList = show . last

-- An action that performs the action of grabbin the files in the directory "."
grabFilePathIO :: IO [FilePath]
grabFilePathIO = listDirectory "."

-- An action that takes as input a list of file paths and writes the first element to the the output and returns ()
printHeadFilePathList :: [FilePath] -> IO ()
printHeadFilePathList = putStrLn . headFilePathList

-- An action that takes as input a list of file paths and writes the last element to the the output and returns ()
printLastFilePathList :: [FilePath] -> IO ()
printLastFilePathList = putStrLn . lastFilePathList

-- checking the types of the new actions
:t grabFilePathIO
:t printHeadFilePathList
:t printLastFilePathList

grabFilePathIO :: IO [FilePath]

printHeadFilePathList :: [FilePath] -> IO ()

printLastFilePathList :: [FilePath] -> IO ()

Again we would like to perforn these action by composing them. But this situation is different, since we use inputs in other actions from previous actions we have to chech that everything is type correct. From the types we deduce that we can not compose all three at the same time since the last composition would lead to the problem that the action `printHeadFilePathList` outputs `()` but the next action `printLastFilePathList` needs the type `[FilePath]`. To still make use of the output of the first action in the last action we must make it know to the last one. Achieving this can be done with **anonymous functions** (sometimes called a lambda expression from the symbolic resemblance with a greek letter lambda (λ) and it ties to lambda calculus). These lambda functions are inline defined function which are imideatly applied to an input. They take the form

In [2]:
(\x -> 2*x + 1) 1

3

Using this we can combine the action while making the inputs **progressivly** available to the other actions in line by traversingly adding the actions from last to first to an encompasing anonymous function. To make sense of this long difficult sentence lets have a look at the following example

In [12]:
import System.Directory (listDirectory)

-- A new expression that is of type IO ()
someIOType :: IO ()
someIOType = putStrLn "Hello" 

-- A pure function that takes as input a list of file paths and return the first one as a string
headFilePathList :: [FilePath] -> String
headFilePathList = show . head

-- A pure function that takes as input a list of file paths and return the first one as a string
lastFilePathList :: [FilePath] -> String
lastFilePathList = show . last

-- An action that performs the action of grabbin the files in the directory "."
grabFilePathIO :: IO [FilePath]
grabFilePathIO = listDirectory "."

-- An action that takes as input a list of file paths and writes the first element to the the output and returns ()
printHeadFilePathList :: [FilePath] -> IO ()
printHeadFilePathList = putStrLn . headFilePathList

-- An action that takes as input a list of file paths and writes the last element to the the output and returns ()
printLastFilePathList :: [FilePath] -> IO ()
printLastFilePathList = putStrLn . lastFilePathList

main = grabFilePathIO >>= (\ x1 -> printHeadFilePathList x1 >>= (\ x2 -> printLastFilePathList x1 ))

-- performing the expression and checking the type of the new expression
main
:t main

"2.2-Creating-Types.ipynb"
".ipynb_checkpoints"

main :: IO ()

Here we first forward the output of the `grabFileIO` action into the first anonymous function which enclosed all other following actions. This anonymous function receives the input in the anonymous variable `x1` which is used in the action `printHeadFilePathList` to output the first element of the list. This action in its turn passes its return value `()` to the second and last anonymous function which labels this input as `x2`. This last anonymous function in its turn does not use this variable `x2` but uses the `x1` varibale to perform the `printLastFilePathList` action. Note that with this construction all inputs gradually become available for the last function. Also observe that `main` has type `IO ()` and is a action on itself, we achieved composition of actions. Now clearly this construction is very cumbersome and not practical in real life coding.

Thats why to make it easier this whole construction is done by a **do block** which is just some syntactic sugar. This block looks just like an imperitave way of programming but importantly, in its core it is still functional. An example of the above composition of actions in a do block becomes

In [14]:
import System.Directory (listDirectory)

-- A new expression that is of type IO ()
someIOType :: IO ()
someIOType = putStrLn "Hello" 

-- A pure function that takes as input a list of file paths and return the first one as a string
headFilePathList :: [FilePath] -> String
headFilePathList = show . head

-- A pure function that takes as input a list of file paths and return the first one as a string
lastFilePathList :: [FilePath] -> String
lastFilePathList = show . last

-- An action that performs the action of grabbin the files in the directory "."
grabFilePathIO :: IO [FilePath]
grabFilePathIO = listDirectory "."

-- An action that takes as input a list of file paths and writes the first element to the the output and returns ()
printHeadFilePathList :: [FilePath] -> IO ()
printHeadFilePathList = putStrLn . headFilePathList

-- An action that takes as input a list of file paths and writes the last element to the the output and returns ()
printLastFilePathList :: [FilePath] -> IO ()
printLastFilePathList = putStrLn . lastFilePathList

main :: IO ()
main = do x1 <- grabFilePathIO
          x2 <- printHeadFilePathList x1 -- We omit the x2 <- here since this action outputs an () and we do not use it.
          printLastFilePathList x1
          
-- perform the new main action
main

"2.2-Creating-Types.ipynb"
".ipynb_checkpoints"

main :: IO ()

## Recap

In this section we have introduced the concepts of IO actions. 

- Haskell knows two types of functions, pure and impure function, the latter are called IO actions. 
- These types of impure fuunction are wrapped and marked by `IO` and may perform usefull side effects. 
- once a type is wrapped in an IO action it can never shed this wrapper. 
- IO action can be composed using two operators, the then operator denoted by `>>` and the bind operator denoted by `>>=`.
- The do block lets you easily compose multiple action withour worring about type correctnes and anonymous functions.